### Linear Regression with multiple feature vector

Continuing from that last notebook with single feature [LinearRegression](https://github.com/kaichogami/machine_learning/blob/master/linear_regression/single_variable.py)(if you haven't read it, go through it!), we will now implement a LinearRegression class which will handle multiple feature vector. We will also solve it through analytical method. Generally analytical method is faster when `n < 10000`, `n` being the number of features, but gets slow and takes a lot of memory if `n` gets greater. Gradiant descent is preferred in such case. Since our code doesn't implement parallelization, our code won't be practical in large data set. Finally we will compare our `true y` and `predicted y` using a scoring method.

### Theory
In the last notebook we worked around with a single feature vector and drew a plot of straight line. Our equation was
$$y = w_0 + w_1X$$
`w1` is our coefficient and `w0` is the intercept. Extending the same idea, where our y is a linear combination of many variables(features) we can re-write our equation as
$$y = w_0 + w_1X_1 + w_2X_2 + .... + w_nX_n$$
where `X1`... `Xn` is our features and `w0`...`wn` is our weights. We will apply gradiant descent algorithm to optimise the same cost function.
$$J(\theta_0, \theta_1, ...) = \sum_{i=1}^n{(y_i - \theta_0 + \theta_1X_1^{(i)} + \theta_2X_2^{(i)}+ ..... +\theta_mX_m^{(i)})^2}$$

For simplicity
$$J(\theta_0) = \sum_{i=1}^n{(y_i - \theta_0 + \theta_1X_1^{(i)} + \theta_2X_2^{(i)}+.....+\theta_mX_m^{(i)})^2}$$

#### Matrix representation
Before moving on, its important to understand the notation of matrices. 
$$X_1^{(i)}$$
Lets look at the above variable. Capital letter indicates its a vector or a matrix. The subscript 1 indicates its the first feature in our matrix. Features are usually the columns and the rows are the examples. Let
$$A \in \mathbb{R}^{2*3}$$
And the matrix be
$$\begin{bmatrix}
1 && 2 && 3 \\
4 && 5 && 6\end{bmatrix}$$
This matrix has a dimension of 2 * 3, 2 rows and 3 columns. Rows are the instances of the data and columns are the features. So `A` has 3 features. Our equation above will then become  
$$y = w_0 + w_1X_1 + w_2X_2 + w_3X_3$$

Also

$$X_1^{(i)}$$
Represents the ith example of the first feature. Watch [this](https://youtu.be/j-MP6CDJiJM?list=PLnnr1O8OWc6asSH0wOMn5JjgSlqNiK2C4) Andrew's video for more clear understanding.

#### Optmisation
Similar to our last notebook, we will use gradiant descent to minimise our cost function. We will find our weights by differentating each wrt each weight, and using the slope in our algorithm. Our equation earlier(single feature) was
$$\frac{\delta}{\delta\theta_1} = \frac{2 \sum_{i=1}^n{((\theta_0 + \theta_1X_i)}X_i - y_i)} {N}$$

and

$$\frac{\delta}{\delta\theta_0} = \frac{2 \sum_{i=1}^n{(\theta_0 + \theta_1X_i} - y_i)} {N}$$

Where `X_i` was the ith example of the first feature. Extending the same idea, we can find weight the `j_th` weight with the equation

$$\theta_j = \theta_j - \alpha \frac{2}{m} \sum_{i=1}^m{(h_\theta(X^{(i)}) - y^{(i)})}X_j^{(i)}$$

#### Analytical
Instead of all the hassle to use the cost function and optimising it, we can simply find the weights by equating the slope of the derivative to the zero. This requires no choosing of learning rate, and no need of deciding the stopping criteria. We will not derive the equations here as they are beyond the scope of this notebook. We will use the derived matrix equation to find the weights.

$$\theta = {(X^TX)^{-1})X^Ty}$$

Notice that we require to find the inverse of a matrix, which has a time complexity of `O(n^3)`. This is the biggest disadvantage of this method. When the features are very very large this becomes extremely slow, in which case we use gradiant descent. I haven't tested out it out fully yet, but with 1000 features this works pretty well. I would choose this method over gradiant descent if say n < 10000 (I am just repeating what Andrew said in his video here).

Now that both methods are clear, lets move on with the coding part. The code is hosted [here](https://github.com/kaichogami/machine_learning/blob/master/linear_regression/multi_variables.py).

In [3]:
import time
import numpy as np

from sklearn.preprocessing import StandardScaler
from multi_variables import MultiLinearRegression

X = np.random.randint(1, 100, [1000, 200]).astype(np.float)
y  = [sum(X[i]) * float(np.random.random_integers(1, 1000, 1)) + 1000
                    for i in xrange(1000)]
sc = StandardScaler().fit(X)
X = sc.transform(X)

mlr = MultiLinearRegression(algorithm='gradiant', alpha=0.001, max_iter=300)
start_gradiant = time.time()
mlr.fit(X, y)
stop_gradiant = time.time()
y_bar_gradiant = mlr.transform(X)

mlr = MultiLinearRegression(algorithm='analytic')
start_analytic = time.time()
mlr.fit(X, y)
stop_analytic = time.time()
y_bar_analytic = mlr.transform(X)

print("Time taken for gradiant to fit is {0} seconds").format(stop_gradiant - start_gradiant)
print("Time taken for analytic method to fit is {0} seconds").format(stop_analytic - start_analytic)

# Calculate the score of our prediction.
def score(y, y_bar):
    return 1 - ((y-y_bar).var() / np.var(y))

print("Score for gradiant method is {0}").format(score(y, y_bar_gradiant))
print("Score for analytical method is {0}").format(score(y, y_bar_analytic))

Time taken for gradiant to fit is 19.4937679768 seconds
Time taken for analytic method to fit is 0.0166280269623 seconds
Score for gradiant method is 0.197290902174
Score for analytical method is 0.200896572395


A lot has happened in the above code. We first import the relevant packges. `StandardScaler` is a scikit-learn package, which standarizes the code by removing the mean. [read here](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) for more details. We then generate our input variable X with a length of 1000 and 200 features. `y` is the label for our data, which is in some way related to X. Its a very bad relation, forgive me for that. The rest is self explainatory, we fit our data first with our model and transform our input matrix to `y` and save it `y_bar`. 
Finally we evaluate our score or how well our model performs. Higher the value, more better it is. The scoring equation is

$$score = 1 - \frac{Var(y - \bar{y})}{Var(y)}$$

Also note the time taken to fit the data by both the algorithms. `Gradiant descent` algorithm seems to be performing very badly. This is because we have not optimised the algorithm(parallelism, numpy functions) in the code.
This ends the notebook with a brief introduction to multiple feature LinearRegression. Thank you for reading!